In [1]:

%%bash
# 定义 proxy_off 函数
function proxy_off() {
    unset http_proxy
    unset https_proxy
    echo -e "已关闭代理"
}

# 定义 proxy_on 函数
function proxy_on() {
    export no_proxy="localhost,127.0.0.1,localaddress,.localdomain.com"
    export http_proxy="http://127.0.0.1:8118"
    export https_proxy=$http_proxy
    echo -e "已开启代理"
}

# 测试函数
proxy_on

已开启代理


In [2]:
! ping www.google.com

PING www.google.com (198.18.0.228): 56 data bytes
64 bytes from 198.18.0.228: icmp_seq=0 ttl=64 time=0.280 ms
64 bytes from 198.18.0.228: icmp_seq=1 ttl=64 time=0.219 ms
64 bytes from 198.18.0.228: icmp_seq=2 ttl=64 time=0.234 ms
64 bytes from 198.18.0.228: icmp_seq=3 ttl=64 time=0.455 ms
64 bytes from 198.18.0.228: icmp_seq=4 ttl=64 time=0.265 ms
64 bytes from 198.18.0.228: icmp_seq=5 ttl=64 time=0.311 ms
^C


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
batch_hsc_cutouts_retry_v3.py
  · 已存在 FITS 跳过
  · 下载失败自动重试
  · 每次下载超时 15 s；失败后等待 5 s
  · 用 --max-connections 1 代替旧脚本不支持的 --threads
"""

from __future__ import annotations
import os, shlex, subprocess, time
from pathlib import Path
from typing import List
import shutil

import pandas as pd

# ------------------ 账号 ------------------
HSC_USR = os.getenv("HSC_USR", "zzhe")
HSC_PWD = os.getenv("HSC_PWD", "9aoO8ZfYKcKSRax54Gx0FXcTX0tBYvak6RO5bV+D")

# ------------------ 路径与常量 ------------------
CSV_PATH   = "/Users/hezizhao/Downloads/jupython-workspace/FH2/crossmatched__withDESI_6''_imgsrc.csv"
SCRIPT     = Path("./data-access-tools/pdr3/downloadCutout/downloadCutout.py")
OUT_DIR    = Path("/Users/hezizhao/Downloads/jupython-workspace/FH2/hsc_cutouts"); OUT_DIR.mkdir(exist_ok=True)
OLD_DIR=Path("/Users/hezizhao/Downloads/jupython-workspace/FH2/hsc_cutouts-old")

FILTERS: List[str] = ["g", "r", "i"]
RERUN       = "pdr3_wide"
FOV_AS      = 3.0
SIZE_DEG    = FOV_AS / 3600.0

MAX_RETRY   = 2
RETRY_WAIT  = 1          # ← 等待 5 s
DL_TIMEOUT  = 15         # ← 每次下载超时 15 s

# ------------------ 读取目标 ------------------
df = pd.read_csv(CSV_PATH,dtype={"TARGETID": "int64"} )
targets = (
    df[df["image source"] == "HSC"][["TARGETID", "TARGET_RA", "TARGET_DEC"]]
    .dropna()
    .reset_index(drop=True)
)
print(f"目标共 {len(targets)} 个，每目标需下载 {len(FILTERS)} 张")



目标共 285 个，每目标需下载 3 张


In [7]:
# ------------------ 主循环 ------------------
idx=0
for row in targets.itertuples(index=True):
    tid = int(row.TARGETID)
    ra, dec = row.TARGET_RA, row.TARGET_DEC
    idx+=1
    for flt in FILTERS:
        old_name = OLD_DIR/ f"hsc{tid}_{flt}"
        old_name = old_name.with_suffix(".fits")
        
        prefix   = OUT_DIR / f"hsc{tid}_{flt}"
        fitsfile = prefix.with_suffix(".fits")
        
        if old_name.exists() and old_name.stat().st_size > 0:
            # print(f"[{idx:3d}] {fitsfile.name} 已存在于旧目录 → 直接复制")
            shutil.copy(old_name,fitsfile)
            continue

        # -- 已有文件跳过 --
        if fitsfile.exists() and fitsfile.stat().st_size > 0:
            # print(f"[{idx:3d}] {fitsfile.name} 已存在 → 跳过")
            continue

        cmd = (
            f"python {SCRIPT} "
            f"--ra {ra:.6f} --dec {dec:.6f} "
            f"--sw {SIZE_DEG:.7f} --sh {SIZE_DEG:.7f} "
            f"--filter {flt} "
            f"--rerun {RERUN} "
            "--max-connections 1 "
            "--image true --mask false --variance false "
            "--type coadd "
            f"--name {prefix} "
            f"--user {HSC_USR} --password {HSC_PWD}"
        )
        cmd_args = shlex.split(cmd)

        # -- 自动重试 --
        for attempt in range(1, MAX_RETRY + 1):
            try:
                subprocess.run(cmd_args, check=True, timeout=DL_TIMEOUT)
                print(f"[{idx+1:3d}] {fitsfile.name} 下载完成 ✓")
                break
            except subprocess.TimeoutExpired:
                err_msg = "timeout"
            except subprocess.CalledProcessError as e:
                err_msg = f"exit {e.returncode}"

            if attempt < MAX_RETRY:
                print(
                    f"[{idx+1:3d}] {fitsfile.name} 第 {attempt} 次失败（{err_msg}），{RETRY_WAIT}s 后重试…"
                )
                time.sleep(RETRY_WAIT)
            else:
                print(
                    f"[{idx+1:3d}] {fitsfile.name} 重试 {MAX_RETRY} 次仍失败 ✗ ({err_msg})"
                )


[ 64] hsc2789831897776129_r.fits 下载完成 ✓
[ 64] hsc2789831897776129_i.fits 下载完成 ✓
[247] hsc39627863657286622_i.fits 下载完成 ✓
[254] hsc39627872750534843_r.fits 下载完成 ✓


Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/urllib/request.py", line 1348, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/opt/anaconda3/lib/python3.11/http/client.py", line 1298, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/opt/anaconda3/lib/python3.11/http/client.py", line 1344, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/opt/anaconda3/lib/python3.11/http/client.py", line 1293, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/opt/anaconda3/lib/python3.11/http/client.py", line 1052, in _send_output
    self.send(msg)
  File "/opt/anaconda3/lib/python3.11/http/client.py", line 990, in send
    self.connect()
  File "/opt/anaconda3/lib/python3.11/http/client.py", line 1470, in connect
    self.sock = self._context.wrap_socket(self.sock,
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/

[262] hsc39627884767219584_g.fits 第 1 次失败（exit 1），1s 后重试…
[262] hsc39627884767219584_g.fits 下载完成 ✓
[264] hsc39627887153774699_r.fits 下载完成 ✓
[264] hsc39627887153774699_i.fits 下载完成 ✓


In [2]:
problem_list=[2789831897776129,39627863657286622,39627887153774699]

In [9]:
tid=problem_list[2]
ra=targets[targets["TARGETID"]==tid]["TARGET_RA"].tolist()[0]
dec=targets[targets["TARGETID"]==tid]["TARGET_DEC"].tolist()[0]
print(tid,ra,dec)

39627887153774699 346.98603246590204 3.935137771804215


In [10]:
idx=0
for flt in FILTERS:
    old_name = OLD_DIR/ f"hsc{tid}_{flt}"
    old_name = old_name.with_suffix(".fits")
    
    prefix   = OUT_DIR / f"hsc{tid}_{flt}"
    fitsfile = prefix.with_suffix(".fits")
    
    if old_name.exists() and old_name.stat().st_size > 0:
        # print(f"[{idx:3d}] {fitsfile.name} 已存在于旧目录 → 直接复制")
        shutil.copy(old_name,fitsfile)
        continue

    # -- 已有文件跳过 --
    if fitsfile.exists() and fitsfile.stat().st_size > 0:
        # print(f"[{idx:3d}] {fitsfile.name} 已存在 → 跳过")
        continue

    cmd = (
        f"python {SCRIPT} "
        f"--ra {ra:.6f} --dec {dec:.6f} "
        f"--sw {SIZE_DEG:.7f} --sh {SIZE_DEG:.7f} "
        f"--filter {flt} "
        f"--rerun {RERUN} "
        "--max-connections 1 "
        "--image true --mask false --variance false "
        "--type coadd "
        f"--name {prefix} "
        f"--user {HSC_USR} --password {HSC_PWD}"
    )
    cmd_args = shlex.split(cmd)

    # -- 自动重试 --
    for attempt in range(1, MAX_RETRY + 1):
        try:
            subprocess.run(cmd_args, check=True, timeout=DL_TIMEOUT)
            print(f"[{idx+1:3d}] {fitsfile.name} 下载完成 ✓")
            break
        except subprocess.TimeoutExpired:
            err_msg = "timeout"
        except subprocess.CalledProcessError as e:
            err_msg = f"exit {e.returncode}"

        if attempt < MAX_RETRY:
            print(
                f"[{idx+1:3d}] {fitsfile.name} 第 {attempt} 次失败（{err_msg}），{RETRY_WAIT}s 后重试…"
            )
            time.sleep(RETRY_WAIT)
        else:
            print(
                f"[{idx+1:3d}] {fitsfile.name} 重试 {MAX_RETRY} 次仍失败 ✗ ({err_msg})"
            )

[  1] hsc39627887153774699_g.fits 下载完成 ✓
[  1] hsc39627887153774699_r.fits 下载完成 ✓
[  1] hsc39627887153774699_i.fits 下载完成 ✓


### 今天弄清楚了，这三个系统就是有问题。2789831897776129和39627887153774699都是只有G波段的。39627863657286622虽然可以下载到r波段，但是是无效图像。39627863657286622没有i波段